In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

ISBN = 9789865321703  # 少男少女見學中 : 日本時代修學旅行開箱

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
driver = webdriver.Chrome(options=my_options)
driver.get("https://lib.moc.gov.tw/F")

time.sleep(1)  # 為了等待網頁加載
select = Select(driver.find_element_by_name("x"))
select.select_by_visible_text(u"ISBN")

search_input = driver.find_element_by_name("y")
search_input.send_keys(ISBN)
# search_input.submit()  # 不知道為什麼無法 submit()？
submit_input = driver.find_element_by_name("Search")
submit_input.click()

click = driver.find_element_by_xpath("/html/body/table[9]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/a")
click.click()

html_text = driver.page_source
dfs = pd.read_html(html_text, encoding="utf-8")
df_ntm = dfs[11]
# driver.quit()

# 清理資料
df_ntm = df_ntm.drop(df_ntm.columns[[0, 1, 5, 6, 8, 9, 10]], axis=1)

for i in range(df_ntm.shape[0]):
    df_ntm.at[i, "到期日"] = df_ntm.at[i, "到期日"].split(";")[-1].strip()

df_ntm.rename(columns={"館藏地": "圖書館",
                        "典藏區域": "館藏地",
                        "到期日": "館藏狀態"}, inplace=True)
df_ntm["圖書館"] = df_ntm["圖書館"].str.replace("臺史博", "國立臺灣博物館")
df_ntm

,圖書館,館藏地,索書號,館藏狀態
0,國立臺灣博物館,一般圖書區,733.26 4431 2020,仍在館內
1,國立臺灣博物館,一般圖書區,733.26 4431 2020 c.1,20210510
2,國立臺灣博物館,一般圖書區,733.26 4431 2020 c.2,仍在館內
